# PPE Detection - YOLO11

In [ ]:
import os, glob, cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
%matplotlib inline

In [ ]:
# model_path = 'ppe_training/yolo11_balanced/weights/best.pt'
# model_path = 'ppe_training/yolo11_subset_run/weights/best.pt'

# Modelul nou antrenat pe Colab (copiază best.pt în folderul proiectului)
model_path = 'best.pt' 

if not os.path.exists(model_path):
    # Fallback la modelul vechi dacă nu găsim best.pt
    print("Nu am găsit best.pt în root, încerc modelul subset...")
    model_path = 'ppe_training/yolo11_subset_run/weights/best.pt'

if not os.path.exists(model_path): model_path = '../' + model_path

print(f"Încarc modelul din: {model_path}")
model = YOLO(model_path)
print(f'Clase: {list(model.names.values())}')

In [ ]:
# --- COD ACTUALIZAT PENTRU DETECTIE MANUSI ---
import glob
import os

# Căutăm specific imagini care conțin MĂNUȘI (clasa 0) în etichete
valid_labels_path = 'datasets/ppe_balanced/valid/labels'
valid_images_path = 'datasets/ppe_balanced/valid/images'

glove_images = []
# Iterăm prin fișierele de etichete
label_files = glob.glob(os.path.join(valid_labels_path, '*.txt'))
print(f"Caut imagini cu mănuși în {len(label_files)} fișiere de etichete...")

for label_file in label_files:
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    has_glove = False
    for line in lines:
        # Verificăm dacă linia începe cu clasa 0 (glove)
        if line.strip().startswith('0 '): 
            has_glove = True
            break
    
    if has_glove:
        # Construim calea către imaginea corespunzătoare
        image_name = os.path.basename(label_file).replace('.txt', '.jpg')
        image_path = os.path.join(valid_images_path, image_name)
        if os.path.exists(image_path):
            glove_images.append(image_path)
    
    if len(glove_images) >= 5: # Ne oprim după ce găsim 5 imagini cu mănuși
        break

test_images = glove_images
if not test_images:
    print("Nu am găsit imagini cu mănuși în validare. Folosesc demo_images.")
    test_images = sorted(glob.glob('demo_images/*.jpg'))[:5]

print(f"Testez pe {len(test_images)} imagini care ȘTIM că au mănuși.")

# Setăm confidența la 0.15 (rezonabil)
results = model(test_images, conf=0.15) 

for i, r in enumerate(results):
    print(f'\nImagine: {os.path.basename(test_images[i])}')
    if len(r.boxes) == 0:
        print("  Nimic detectat.")
    for box in r.boxes:
        cls_id = int(box.cls[0])
        cls_name = model.names[cls_id]
        conf = float(box.conf[0])
        print(f'  {cls_name} ({conf:.2f})')